In [7]:
# Import necessary libraries
import pandas as pd
import numpy as np
from google.colab import files

# STEP 1: Upload Only Required Files
print("Please upload 'inventory_dataset.csv' and 'buyer_preferences.xlsx'")
uploaded = files.upload()

# STEP 2: Load Cleaned Inventory Dataset and Buyer Preferences
inventory_df = pd.read_csv("inventory_dataset.csv")
buyer_prefs = pd.read_excel("buyer_preferences.xlsx")

# STEP 3: Standardize Column Names
def clean_column_names(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace(r"[^a-z0-9_]", "", regex=True)
    return df

inventory_df = clean_column_names(inventory_df)
buyer_prefs = clean_column_names(buyer_prefs)

# STEP 4: Print Column Names for Debugging
print("Inventory Dataset Columns:", inventory_df.columns.tolist())
print("Buyer Preferences Columns:", buyer_prefs.columns.tolist())

# STEP 5: Map Incorrect Column Names to Expected Ones
column_mapping = {
    "material_quality_norm": "grade",
    "surface_coating": "finish",
    "nenndicke_nnnnn_mm_mit_dezimalpunkt": "thickness_mm",
    "width_mm": "width_mm",
    "gewicht_kg": "gross_weight_kg",
    "number_of_coils": "quantity",
}

# Rename columns in inventory_dataset.csv
inventory_df = inventory_df.rename(columns=column_mapping)

# STEP 6: Check If All Required Columns Exist
required_columns = ['grade', 'finish', 'thickness_mm', 'width_mm', 'gross_weight_kg', 'quantity']
missing_columns = [col for col in required_columns if col not in inventory_df.columns]

if missing_columns:
    print(f"Missing Columns in Inventory Dataset AFTER renaming: {missing_columns}")
    raise KeyError(f"Missing required columns in inventory dataset: {missing_columns}")

# STEP 7: Convert Numeric Columns to Proper Format
numeric_columns = ['preferred_thickness_mm', 'preferred_width_mm', 'max_weight_kg', 'min_quantity']
for col in numeric_columns:
    if col in buyer_prefs.columns:
        buyer_prefs[col] = pd.to_numeric(buyer_prefs[col], errors='coerce')

# STEP 8: Matching Logic - Finding Best Supplier for Each Buyer
recommendations = []

for _, buyer in buyer_prefs.iterrows():
    if not all(col in buyer_prefs.columns for col in ['preferred_grade', 'preferred_finish', 'preferred_thickness_mm', 'preferred_width_mm', 'max_weight_kg', 'min_quantity']):
        print(f"Missing required buyer preference columns. Skipping buyer ID {buyer.get('buyer_id', 'Unknown')}.")
        continue

    matching_suppliers = inventory_df[
        (inventory_df['grade'].notna()) & (inventory_df['grade'] == buyer['preferred_grade']) &
        (inventory_df['finish'].notna()) & (inventory_df['finish'] == buyer['preferred_finish']) &
        (inventory_df['thickness_mm'].notna()) & (inventory_df['thickness_mm'] == buyer['preferred_thickness_mm']) &
        (inventory_df['width_mm'].notna()) & (inventory_df['width_mm'] == buyer['preferred_width_mm']) &
        (inventory_df['gross_weight_kg'].notna()) & (inventory_df['gross_weight_kg'] <= buyer['max_weight_kg']) &
        (inventory_df['quantity'].notna()) & (inventory_df['quantity'] >= buyer['min_quantity'])
    ]

    for _, row in matching_suppliers.iterrows():
        recommendations.append({
            'buyer_id': buyer['buyer_id'],
            'recommended_material': row.get('material_name', 'N/A'),
            'supplier_grade': row['grade'],
            'supplier_finish': row['finish'],
            'supplier_thickness_mm': row['thickness_mm'],
            'supplier_width_mm': row['width_mm'],
            'supplier_weight_kg': row['gross_weight_kg'],
            'supplier_quantity': row['quantity']
        })

# Convert recommendations to DataFrame
recommendation_df = pd.DataFrame(recommendations)

# STEP 9: Save and Download the Recommendation Table
recommendation_df.to_csv("recommendation_table.csv", index=False)
print("Task 3 Completed: Recommendation table saved as 'recommendation_table.csv'")

# STEP 10: Download the Cleaned Recommendation Dataset (Google Colab Only)
files.download("recommendation_table.csv")

# STEP 11: Display Preview of Recommendations
recommendation_df.head()


Please upload 'inventory_dataset.csv' and 'buyer_preferences.xlsx'


Saving buyer_preferences.xlsx to buyer_preferences (6).xlsx
Saving inventory_dataset.csv to inventory_dataset (5).csv
Inventory Dataset Columns: ['werksgte', 'bestellgtentext', 'nenndicke_nnnnn_mm_mit_dezimalpunkt', 'breite', 'lnge', 'gewicht_kg', 'cluster', 'sigehalt', 'mngehalt', 'pgehalt', 'sgehalt', 'crgehalt', 'nigehalt', 'mogehalt', 'vgehalt', 'cugehalt', 'nbgehalt', 'tigehalt', 'algehalt', 'bgehalt', 'streckgrenze', 'zugfestigkeit', 'dehnung', 'product_type', 'order_id', 'site', 'material_name', 'material_number', 'material_quality_norm', 'surface_coating', 'defect_notes', 'nominal_thickness_mm', 'width_mm', 'length_mm', 'height_mm', 'mass_min_kg', 'number_of_coils', 'delivery_earliest', 'delivery_latest', 'inco_term', 'buy_now_eur_per_ton', 'minmax_bid_eur_per_ton', 'co2_per_ton_max_kg', 'valid_until']
Buyer Preferences Columns: ['buyer_id', 'preferred_grade', 'preferred_finish', 'preferred_thickness_mm', 'preferred_width_mm', 'max_weight_kg', 'min_quantity']
Task 3 Completed: 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

""
